<div style='text-align: right; font-size: 16px;'>
<br/>
<img src="images/HWR_logo.svg" alt="drawing" width="30%"/>
<br/>
Fachbereich Duales Studium Wirtschaft • Technik<br/>
ET1031 Mathematische Grundlagen III (Signale und Systeme)<br/>
Prof. Dr. Luis Fernando Ferreira Furtado, Berlin, 30.10.2025<br/>
</div>

# Kapitel 14. Filter

<hr style="border:solid #000000 1px;height:1px;">

In [ ]:
from scipy import signal
import numpy as np
from si_prefix import si_format
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import plotly.graph_objects as go
from ipywidgets import interact, fixed, IntSlider, ToggleButtons, Select
from IPython.display import Audio, display, Latex
import librosa
import warnings
warnings.filterwarnings("ignore")

def display_TransferFunction(num, den):
    
    def list_to_txt(list_of_values):
        list_of_values = list_of_values[::-1]
        txt = []
        for i in range(len(list_of_values)):      
            temp = []
            if list_of_values[i]!=0:
                if (i!=0) and (list_of_values[i]==1):
                    temp.append('+')
                elif (i!=0) and (list_of_values[i]==-1):
                    temp.append('-')                    
                else:
                    temp.append('{0:+}'.format(list_of_values[i])) 
                if i != 0:
                    temp.append('s')         
                if i >1:
                    temp.append('^' + str(i))
                txt.append(''.join(temp))
        txt = ''.join(txt[::-1] )  
        if txt[0] == '+':
            txt=txt[1:]
        return txt

    eq = r'H(s)=\frac{'+ list_to_txt(num) +'}{'+ list_to_txt(den) +'}'
        
    display(Latex(r'\begin{align}' + eq +'\end{align}')) 


def plot_bode(num, den):
    # num = Übertragungsfunktion-Zähler im Listenformat
    # den = Übertragungsfunktion-Nenner im Listenformat
    
    # Build the systeme with the transfer function
    H = signal.TransferFunction(num, den)

    # Display the Transfer Function
    display_TransferFunction(num, den)

    # Calculate the Bode-Plot parameters
    omega, amp, phase = signal.bode(H)
    
    # Bode-Diagramm
    fig = plt.figure(figsize=(14, 4), facecolor='white')
    gs = GridSpec(nrows=1, ncols=2)    

    # Magnitude (Bode-Plot)
    ax = fig.add_subplot(gs[:, 0])
    ax.semilogx(omega, amp)  
    plt.title('Bode-Diagramm für Amplitude')
    plt.xlabel('Frequenz [rad/s]'); plt.ylabel('Amplitude [dB]')
    plt.margins(0, 0.1)
    plt.grid(which='both', axis='both')
    
    # Phase (Bode-Plot)
    ax = fig.add_subplot(gs[:, 1])
    ax.semilogx(omega, phase)
    plt.title('Bode-Diagramm für Phase')
    plt.xlabel('Frequenz [rad/s]'); plt.ylabel('Phase [°]')
    plt.margins(0, 0.1)
    plt.grid(which='both', axis='both')    
    
    plt.show()

def plot_spectrogram(filename, duration, time=True, mel=True, n_fft=1024, filter={'type':'', 'freq':[]}):
        
    # Load the file in the time domain
    y, sr = librosa.load(filename, duration=duration)
    
    print(filter['type'])
    # Filter the signal    
    if filter['type'] != '':
        try:
            sos = signal.butter(filter['order'], filter['freq'], filter['type'], fs=sr*2, output='sos')
            y = signal.sosfilt(sos, y)
        except:
            pass
    
    # Transform from the time domain to the frequency domain 
    # Short-Time Fourier Transform (STFT) = FFT
    Y = librosa.stft(y, n_fft=n_fft)   
    
    # Trasform the power to DB
    Y = librosa.power_to_db(abs(Y) ** 2, ref=np.max)    
    
    # Transform from the time domain to the frequency domain (Mel-Scale)
    # FFT adatpted to the Mel-Scale
    if mel:
        Ymel = librosa.feature.melspectrogram(y=y, sr=sr, n_fft=n_fft)
        Ymel = librosa.power_to_db(abs(Ymel) ** 2, ref=np.max) 
  
    # Plot signal
    fig = plt.figure(figsize=(14, 3*(time + 1 + mel)), facecolor='white')
    gs = GridSpec(nrows=(time + 1 + mel), ncols=1)    
    n = 0
    
    if time:
        ax = fig.add_subplot(gs[n, :])
        ax.set_title(label='Zeitbereich', loc='left')
        ax.set(xlim=[0, duration + .001])
        librosa.display.waveshow(y, sr=sr, ax=ax)
        n += 1
        
    ax = fig.add_subplot(gs[n, :])
    ax.set(xlim=[0, duration + .001])
    ax.set_title(label='Frequenzbereich', loc='left')
    librosa.display.specshow(Y, sr=sr*2, x_axis='time', y_axis='linear', ax=ax)
    n += 1
    
    if mel:
        ax = fig.add_subplot(gs[n, :])
        ax.set_title(label='Frequenzbereich - Mel-Skala', loc='left')
        ax.set(xlim=[0, duration + .001])
        librosa.display.specshow(Ymel, sr=sr, x_axis='time', y_axis='mel', ax=ax)
    
    plt.show()
    
    if filter['type'] != '':
        filter_text = ' | Filtertyp:[' + filter['type'] + '], Filterfreq.:' + str(filter['freq']) + 'Hz' +\
                      ' , Filterordnung:[' + str(filter['order']) + '.]'
    else:
        filter_text = ' | Ohne Filter'
        
    print(filename + filter_text)
    # Return y and sr for the audio play-button
    return y, sr

#### Digitaler Rauschfilter
<br/>
Signalgeräusche im Zeitbereich sind nur sehr schwer zu erkennen und zu eliminieren. Dennoch ist es möglich, das Signal in den Frequenzbereich zu bringen, nur die dominanten Frequenzen mit einer Toleranzschwelle zu filtern und das gefilterte Signal in den Zeitbereich zurückzubringen.

In [ ]:
freq_1 = 440    # Hz
freq_2 = 555    # Hz
freq_3 = 990    # Hz
samples = 4096 # samples 
filter_threshold = 0.2 # Filter threshold (Grenzwert von 0.0 bis 1.0)

# Time domain
t = np.arange(0, 1, 1/samples)
x_ideal = np.cos(2 * np.pi * freq_1 * t) + np.cos(2 * np.pi * freq_2 * t) + np.cos(2 * np.pi * freq_3 * t) # Clean (pure) input signal x(t)
x_noise = x_ideal + (5 * np.random.randn(len(t))) # Noise (Geräusch) input signal x(t) 

# Fast Fourier Transformation
X_noise = np.fft.fft(x_noise, samples) # Noise input signal represented in the frequency domain X(f)

# Frequency domain
# Extract the power-spectrum with only the real values of X_noise 
X_noise_pwr = abs(X_noise) ** 2 / len(t) # (Spektrale Leistungsdichte)

# Filter
filter = X_noise_pwr > (max(X_noise_pwr) * filter_threshold)  # Filtermatrix
X_ideal_pwr = filter * X_noise_pwr                # Gefilterte Spektrale Leistungsdichte
X_filtered = filter * X_noise             # Gefilterte Fourier-Koeffizienten 
x_filtered = np.fft.ifft(X_filtered).real # Inverse FFT des gefilterten Signals (nur der reale Teil)

# Plot signal in time domain
fig = go.Figure();print()
fig.add_trace(go.Scatter(x=t, y=x_ideal, mode='lines', name='x(t) ideal'))
fig.add_trace(go.Scatter(x=t, y=x_noise, mode='lines', name='x(t) with noise'))
fig.add_trace(go.Scatter(x=t, y=x_filtered, mode='lines', name='x(t) filtered'))
fig.update_layout(height=300, title='Zeitbereich', template='plotly_white', margin=dict(l=20, r=20, t=25, b=20), xaxis=dict(title_text='time [s]'))
fig.show()

# Plot signal in frequency domain
f = np.arange(1, int(samples/2), dtype='int') 
fig = go.Figure();print()
fig.add_trace(go.Scatter(x=f, y=X_ideal_pwr[f], mode='lines', name='X(f) ideal'))
fig.add_trace(go.Scatter(x=f, y=X_noise_pwr[f], mode='lines', name='X(f) with noise'))
fig.add_trace(go.Scatter(x=f, y=X_filtered.real[f], mode='lines', name='X(f) filtered'))
fig.add_trace(go.Scatter(x=f, y=np.ones(len(f))*(max(X_noise_pwr)*filter_threshold), mode='lines', name='filter_threshold'))
fig.update_layout(height=250, title='Frequenzbereich', template='plotly_white', margin=dict(l=20, r=20, t=25, b=20), xaxis=dict(title_text='frequency [Hz]'))
fig.show()

print('x(t) ideal')
display(Audio(x_ideal, rate=samples))
print('x(t) with noise')
display(Audio(x_noise, rate=samples))
print('x(t) filtered')
display(Audio(x_filtered, rate=samples))

<hr style="border:solid #000000 1px;height:1px;">

#### Spektrogramm
<br/>
Das Spektrogramm verschiedener Audioverlust-Kompressionskonfigurationen (z.B. MP3) zeigt, dass einige Frequenzen vermindert werden und andere an Auflösung verlieren.

In [ ]:
def update(compression):
    
    # Plot the spectrogram
    y, sr = plot_spectrogram('data\\Sample_Kapitel_14_' + compression + '.mp3', 15, False, False, 1024)

    # Display audio buttton
    display(Audio(y, rate=sr))

compression = Select(options=['128Kbps_44100Hz', '64Kbps_24000Hz', '32Kbps_16000Hz', '16Kbps_8000Hz'], value='128Kbps_44100Hz', description='MP3:')

interact(update, compression=compression);

<hr style="border:solid #000000 1px;height:1px;">

#### (Mel-Scale)
<br/>
Die Mel Frequency Cepstral Coefficients (MFCC; deutsch Mel-Frequenz-Cepstrum-Koeffizienten) führen zu einer kompakten Darstellung des Frequenzspektrums. Das Mel im Namen beschreibt die wahrgenommene Tonhöhe.<br/>
<br/>
Die Website xeno canto bietet kostenlos Vogelstimmen aus aller Welt für Neugierige an. Die Dateien haben einen Code und den Namen des Vogels, der mit dem Ton verbunden ist. 

In [ ]:
# XC27331_apapan,  Aufzeichner:Daniel Lane
# XC144826_akiapo, Aufzeichner:Eric VanderWerf
# XC473563_barbet, Aufzeichner:Peter Boesman

def update(bird):

    # Plot the spectrogram
    y, sr = plot_spectrogram('data\\' + bird + '.ogg', 6)

    # Display audio buttton
    display(Audio(y, rate=sr))

bird = Select(options=['XC27331_apapan', 'XC144826_akiapo', 'XC473563_barbet'], value='XC27331_apapan', description='Bird:')

interact(update, bird=bird);

<hr style="border:solid #000000 1px;height:1px;">

#### Filter
<br/>

In [ ]:
def update_type(bird, type):

    def update_filter(bird, type, order, freq1, freq2):
        
        if (type == 'bandpass' or type == 'bandstop'):
            if freq2 > freq1:
                freq = [freq1, freq2]
            else:
                freq = [freq2, freq1]
        else:
            freq = [freq1]
            
        y, sr = plot_spectrogram('data\\' + bird + '.ogg', 6, filter={'type':type, 'order':order, 'freq':freq})
        display(Audio(y, rate=sr))

    # Translate the filter type
    type_dict = {'Keine':'', 'Tiefpassfilter':'lowpass', 'Hochpassfilter':'highpass', 'Bandpassfilter':'bandpass', 'Bandsperrfilter':'bandstop'}
    type = type_dict[type]

    # Present slider options according to filter
    if type == 'lowpass' or type == 'highpass':
        bird = fixed(bird)
        type = fixed(type)            
        order = IntSlider(value=1, min=1, max=40, description='Order:')
        freq1 = IntSlider(value=5000, min=0, max=20000, step=100, description='Freq [Hz]:')
        freq2 = fixed(None)    
        interact(update_filter, bird=bird, type=type, order=order, freq1=freq1, freq2=freq2)
    elif type == 'bandpass' or type == 'bandstop':
        bird = fixed(bird)
        type = fixed(type)                    
        order = IntSlider(value=1, min=1, max=40, description='Order:')
        freq1 = IntSlider(value=7000, min=0, max=20000, step=100, description='Freq1 [Hz]:')
        freq2 = IntSlider(value=8000, min=100, max=20000, step=100, description='Freq2 [Hz]:')
        interact(update_filter, bird=bird, type=type, order=order, freq1=freq1, freq2=freq2)
    else:
        bird = fixed(bird)
        type = fixed(type)            
        order = fixed(0)
        freq1 = fixed(None)
        freq2 = fixed(None)    
        interact(update_filter, bird=bird, type=type, order=order, freq1=freq1, freq2=freq2)        
    
    
# First interact: filter type options
type = ToggleButtons(options=['Keine', 'Tiefpassfilter', 'Hochpassfilter', 'Bandpassfilter', 'Bandsperrfilter'], description='Filter:')
bird = Select(options=['XC27331_apapan', 'XC144826_akiapo', 'XC473563_barbet'], description='Bird:')
interact(update_type, type=type, bird=bird);